# Pushing particles (2-D)
Pushing particles around a mesh requires finding a good estimate of the eulerian velocity field on each particle location.

When interpolating onto particle points, we need to know which element each particle is inside of and the barycentric coordinates within that element. As input to such a method, we have 

1. `(xp,yp)` - the longitude and latitude of all particles
2. `(xv,yv,iv)` - mesh vertices and associated indices (also called `nod2d` in FESOM2)
3. `(ev)` - array of size `(1:nelem,1:3)` listing the element to vertex connectivity

To interpolate the mesh information onto the particle positions, we need to be able to identify the element each particle resides within. Once the element ID's are identified, computational coordinates / barycentric weights can be calculated and the `u` and `v` velocity components can be interpolated onto each point. If we are using an explicit time integration scheme, the particles can be advanced to new positions using the interpolated velocity fields. On the subsequent time step, we can check to see if each particle is still within the same element it was last seen in. If a particle is found in the same element, the computational coordinates can be re-calculated; if it is not, then we can search through the element neighbors to identify where the particle has gone to. In general, the algorithm looks like the following

* Initial element search. For each particle, determine the element ID it resides in
* Time loop
  * For each particle, calculate the barycentric weights (input: particle position, particle element IDs, element to vertex mapping )
  * For each particle, interpolate `u` and `v` onto particle positions (input: barycentric weights, particle element IDs, element to vertex mapping, u, v)
  * For each particle, update position
  * For each particle, update element IDs

## Initial element search
For an initial particle distribution, we need to perform a search to determine which element each particle resides in. When searching for the elements in a mesh that particles reside in for interpolation, efficiency and scalability are key. This becomes particularly important when simulating a large number of particles in a high-resolution grid or unstructured mesh. Here are some methods that scale well for finding elements in a mesh:

1. **Direct Grid Indexing (for Structured Grids)** : **\(O(1)\)** 
2. **Bounding Volume Hierarchies (BVH) (for Unstructured Meshes)** : Construction is **\(O(n \log n)\)**, and querying is **\(O(\log n)\)**, making it scalable for large meshes.
3. **KD-Trees (for General Geometries; commonly used in mesh free methods)** : Querying is **\(O(\log n)\)**
4. **Quadtrees** : Querying is **\(O(\log n)\)**
5. **Voronoi Diagrams** : Querying is **\(O(\log n)\)** *if a suitable search structure is employed (e.g., using a Voronoi graph or by combining with KD-tree/BVH structures)*. It’s more computationally expensive set up but can provide fast querying.
6. **Morton/Z-Order Curves (for Space-Filling Curves)** : Querying is **\(O(\log n)\)**; *Morton codes are particularly good for parallelization and GPU-based implementations.*
7. **Spatial Hashing (for Dynamic Particle Systems)** : Querying is **\(O(1)\)**; *Care is needed to avoid collisions and handle edge cases efficiently*

In Particle-In-Cell (PIC) simulations with **unstructured 2D triangular meshes**, determining which mesh element (triangle) a particle resides in is a critical step for accurate interpolation and force computation. **Spatial hashing** can be adapted to accelerate this process, even with the complexity of an unstructured mesh. The main challenge in this case is efficiently searching the large number of mesh elements (triangles) to find the one that contains a given particle, especially as the number of particles increases.

Here’s how spatial hashing can be applied in this context, along with an example.

## Spatial Hashing with an Unstructured 2D Triangular Mesh

1. **Divide the Space into Hash Cells**:
   - Although the underlying mesh is unstructured (i.e., made up of triangles of various shapes and sizes), the overall 2D simulation space can still be divided into a grid of square (or rectangular) **hash cells**. Each hash cell corresponds to a "bucket" in the hash table. Hash cells are typically defined by their lower left corner vertex and the cell size **(\delta x, \delta y)**
   - Each triangle in the unstructured mesh is assigned to one or more hash cells based on its bounding box. This means that each hash cell contains a list of the triangles that overlap with it.

2. **Preprocessing: Assign Triangles to Hash Cells**:
   - For each triangle in the mesh, compute its **axis-aligned bounding box (AABB)**. This is the smallest rectangle that fully encloses the triangle, aligned with the coordinate axes.
   - Using the bounding box, determine which hash cells the triangle overlaps. This might involve checking multiple hash cells (for example, if the triangle spans several cells).
   - Store the triangle’s reference in each of the relevant hash cell buckets in the hash table. A good hash function (as described earlier) is used to map the coordinates of each hash cell to a hash key.

3. **Particle-to-Element Search**:
   - For each particle, compute the hash key of the cell that contains its position.
   - Check all the triangles associated with that hash cell (retrieved from the hash table).
   - For each triangle in the bucket, perform a **point-in-triangle test** to determine if the particle lies inside the triangle.

4. **Point-in-Triangle Test**:
   - For each triangle found in the hash cell, the particle’s position is tested using **barycentric coordinates** or other geometric tests to check whether the particle lies inside the triangle.
   - If the particle lies inside the triangle, it is assigned to that triangle for the purpose of interpolation.

5. **Handling Edge Cases**:
   - If the particle’s position is near a triangle edge or vertex, it may belong to multiple neighboring triangles. In such cases, you may need to apply additional logic to select the triangle or handle interpolation across multiple elements.

In 2D spatial hashing, **hash cells** are typically defined as rectangular or square regions of space. When defining the position of a hash cell, the reference point for computing the hash key can be either:

1. **The hash cell center**, or
2. **One of the four corner nodes** (usually the lower-left corner, but this can be flexible).

## Common Practice: Lower-Left Corner Node
The most common approach is to take **one of the corner nodes** (usually the **lower-left corner**) of the hash cell as the reference point for computing the hash key. This makes the hashing scheme straightforward and easy to compute, especially when working with regularly spaced grids.

### How It Works:
1. **Define the Hash Cell Size**: Suppose each hash cell has a size \(\Delta x \times \Delta y\).
   
2. **Compute the Hash Cell Coordinates**: For a particle at position \((x, y)\), the hash cell it belongs to is determined by calculating the integer coordinates of the hash cell's lower-left corner:
   \[
   i = \left\lfloor \frac{x}{\Delta x} \right\rfloor, \quad j = \left\lfloor \frac{y}{\Delta y} \right\rfloor
   \]
   where \(\left\lfloor \cdot \right\rfloor\) represents the floor function, which rounds down to the nearest integer.
   
3. **Hash Key Computation**: A hash key is then computed based on these integer cell coordinates, often using a hash function like:
   \[
   \text{hash}(i, j) = (i \times p_1) \oplus (j \times p_2) \mod N
   \]
   where \(p_1\) and \(p_2\) are large prime numbers, and \(N\) is the size of the hash table.

By using the lower-left corner as the reference point, the calculation of which cell a particle belongs to is simple, consistent, and unambiguous. This approach is particularly useful when the simulation space is divided into uniform cells, making it easy to map a particle's coordinates directly to the hash cell.

## Alternative Approach: Cell Centers
An alternative but less common approach is to define hash cells based on their **center** points instead of their corner nodes. In this case, the particle's position would be compared to the center of the closest hash cell, and the coordinates of the center would be used in the hash function.

### How It Works:
1. **Define the Hash Cell Centers**: The center of each hash cell is positioned at:
   \[
   x_{\text{center}} = \left( i + 0.5 \right) \Delta x, \quad y_{\text{center}} = \left( j + 0.5 \right) \Delta y
   \]
   
2. **Compute the Nearest Cell Center**: The particle’s position is compared to the nearest cell center, which is determined by:
   \[
   i = \left\lfloor \frac{x}{\Delta x} - 0.5 \right\rfloor, \quad j = \left\lfloor \frac{y}{\Delta y} - 0.5 \right\rfloor
   \]
   
3. **Hash Key Computation**: The hash key is computed similarly based on the cell’s center coordinates.

## Comparison: Corner Nodes vs. Cell Centers
- **Lower-Left Corner Nodes**:
  - Simple and easy to compute.
  - Faster to implement and directly matches the regular grid layout in many simulations.
  - Particularly useful when mesh boundaries and edges align with the grid.
  
- **Cell Centers**:
  - Sometimes useful if your logic is based on distances to the center of cells.
  - However, it requires slightly more complex calculations and may not align as easily with regular grids used in particle-in-cell (PIC) methods.

## Which One to Use?
- **Corner nodes** (particularly the lower-left corner) are the most common choice in spatial hashing for 2D grids because they provide a simple and efficient way to compute the hash keys and directly map a particle's position to a grid cell.
- **Cell centers** are less common and usually come into play in cases where interactions are more naturally defined with respect to distances from the center of a grid cell.

For unstructured meshes (like 2D triangular meshes), **corner node hashing** is generally preferred because it simplifies the mapping between particles and mesh elements, reducing computational overhead.

### Example Walkthrough

Consider an unstructured 2D triangular mesh with triangles of various shapes and sizes, where particles move across the mesh. Here’s an example to illustrate spatial hashing in action.

#### Step 1: Set Up the Hash Table
Suppose the simulation domain is divided into a grid of hash cells, each corresponding to a spatial region in the 2D plane. For simplicity, let's assume we have a grid with cells of size \( \Delta x \times \Delta y \).

|Hash Cell Coordinates| Contains Triangles (IDs)|
|--------------------|-------------------------|
|(0,0)               | [Triangle 1, Triangle 5] |
|(0,1)               | [Triangle 2, Triangle 7] |
|(1,0)               | [Triangle 3, Triangle 6] |
|(1,1)               | [Triangle 4, Triangle 8] |

#### Step 2: Preprocess the Mesh Triangles
For each triangle in the mesh, calculate its bounding box and determine which hash cells it overlaps. For instance:

- **Triangle 1**: Vertices: \((0.1, 0.1)\), \((0.4, 0.2)\), \((0.3, 0.5)\).
    - Bounding box: \([0.1, 0.3]\) in \(x\), \([0.1, 0.5]\) in \(y\).
    - This overlaps with hash cell \((0,0)\).

- **Triangle 2**: Vertices: \((0.4, 0.6)\), \((0.7, 0.7)\), \((0.6, 0.9)\).
    - Bounding box: \([0.4, 0.7]\) in \(x\), \([0.6, 0.9]\) in \(y\).
    - This overlaps with hash cell \((0,1)\).

The hash table is populated with these references, where each cell contains a list of overlapping triangles.

#### Step 3: Particle Search
Suppose a particle is located at position \((0.45, 0.7)\).

1. **Compute Hash Key**:
   - The particle’s position corresponds to hash cell \((0,1)\) based on its coordinates.
   - Using the hash function, retrieve the list of triangles associated with this hash cell. Let’s say the list contains **Triangle 2** and **Triangle 7**.

2. **Point-in-Triangle Test**:
   - Perform a point-in-triangle test for each triangle in the list.
   - For **Triangle 2**, you calculate barycentric coordinates for the particle's position relative to the triangle's vertices.
     - If the barycentric coordinates indicate that the particle is inside Triangle 2, this is the containing element.
   - If not, move on to **Triangle 7** and repeat the test.

3. **Assign the Particle to the Triangle**:
   - If the particle is found to reside in Triangle 2, the particle’s position is associated with Triangle 2, and any necessary interpolation (for fields, charge, etc.) will be performed using the values associated with Triangle 2’s vertices.

#### Step 4: Rehashing for Particle Motion
As particles move across the mesh, they may cross into new hash cells. After each simulation time step, rehash particles based on their new positions, and repeat the process to determine their new containing triangles.

### Benefits of Spatial Hashing in PIC with Unstructured Meshes

1. **Efficiency**: Without spatial hashing, determining the triangle that contains a particle in an unstructured mesh would involve checking every triangle, resulting in a complexity of **\(O(n)\)** per particle, where \(n\) is the number of triangles. Spatial hashing reduces this search to **\(O(1)\)** on average, significantly speeding up the simulation.
   
2. **Scalability**: By mapping the 2D simulation space to a grid, spatial hashing allows for fast querying and efficient handling of large numbers of particles in large meshes, which scales well as the mesh resolution increases.

3. **Parallelization**: The hash table construction and particle search process can be parallelized, allowing for efficient execution on multi-core processors or GPUs.

### Challenges

1. **Hash Cell Size**: Choosing an appropriate cell size is important. Cells that are too large may result in many triangles per cell, slowing down the search. Cells that are too small may increase the number of hash cells a triangle overlaps with, making the hash table more memory-intensive.
   
2. **Edge Cases**: Particles near triangle edges or vertices may require additional checks to handle ambiguity in determining the containing element.
